In [ ]:
import os
import warnings

import numpy as np
import pandas as pd

import neurokit2 as nk

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from hmmlearn import hmm

# Suppress warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Sliding Window Hidden Markov Model

---
## Constructing Observations

### Grab the physiological timestamps / takeover times

In [ ]:
processed_physio_folder_path = "./Physiological Preprocessed/"

exp2_folder_path = processed_physio_folder_path + "Exp2"

In [ ]:
exp2_takeover_times = pd.read_csv(
    "./AdVitam/Exp2/Preprocessed/Physio and Driving/timestamps_obstacles.csv"
)
exp2_takeover_times.iloc[:, 2:] = exp2_takeover_times.iloc[:, 2:].apply(pd.to_timedelta, unit="s")
exp2_takeover_times["subject_id"] = exp2_takeover_times["subject_id"].apply(
    lambda x: x.split("T")[0] + "T" + x.split("T")[1].zfill(2)
)
exp2_takeover_times["subject_id"] = exp2_takeover_times["subject_id"].astype(str)
exp2_takeover_times.drop(columns=["label_st"], inplace=True)
exp2_takeover_times.sort_values(by=["subject_id"], inplace=True)

for column in exp2_takeover_times.columns:
    if "TrigObs" in column:
        exp2_takeover_times = exp2_takeover_times.rename(
            columns={column: column.replace("TrigObs", "") + "TOR"}
        )
    elif "RepObs" in column:
        exp2_takeover_times = exp2_takeover_times.rename(
            columns={column: column.replace("RepObs", "Response")}
        )

exp2_takeover_times

### Create observations for takoever and driving segments

In [ ]:
def collect_observations(exp2_folder_path, columns_to_drop, deviation_columns, columns_to_normalize, window_size, step_size):
    """
    Create the observations for the slow and fast takeover times.

    Args:
        phsyiological_data_dictionary (dict): A dictionary containing the segmented physiological data files.
        takeover_times (pd.DataFrame): A DataFrame containing the takeover times.
        driver_demographic_data (pd.DataFrame): A DataFrame containing the driver demographic data.
        window_length (int): The length of the window in minutes.
        window_step (int): The step size for the window
        step_sizes (list): A list of step sizes for the window.
        tot (str): The threshold for the takeover time.

    Returns:
        list: A list of observations for the slow takeover times.
        list: A list of observations for the fast takeover times.
    """

    driving_observations_data = []
    takeover_observations_data = []

    # Exp2
    phsyiological_data_dictionary = {}
    for file in os.listdir(exp2_folder_path):
        # Split the file name into the participant and period
        f = file.split("_")
        participant = f[0]
        period = f[1].split(".")[0]

        if period != "baseline" and period != "driving":
            continue

        # Baseline Data
        if participant not in phsyiological_data_dictionary:
            phsyiological_data_dictionary[participant] = {}
            phsyiological_data_dictionary[participant][period] = pd.read_csv(
                exp2_folder_path + "/" + file
            )

        # Physiological data
        else:
            phsyiological_data_dictionary[participant][period] = pd.read_csv(
                exp2_folder_path + "/" + file
            )

            # Process the physiological data
            baseline_physio = phsyiological_data_dictionary[participant]["baseline"].copy()
            del phsyiological_data_dictionary[participant]["baseline"]
            baseline_physio.Time = pd.to_timedelta(baseline_physio.Time)
            baseline_physio = baseline_physio.set_index("Time")

            experiment_physio = phsyiological_data_dictionary[participant]["driving"].copy()
            del phsyiological_data_dictionary[participant]["driving"]
            experiment_physio.Time = pd.to_timedelta(experiment_physio.Time)
            experiment_physio = experiment_physio.set_index("Time")

            # print(participant)

            # Calculate the deviation from the baseline mean
            baseline_physio_mean = baseline_physio[deviation_columns].mean()
            baseline_deviation = experiment_physio[deviation_columns] - baseline_physio_mean
            experiment_physio.drop(
                columns=deviation_columns,
                inplace=True,
            )
            experiment_physio = pd.concat([baseline_deviation, experiment_physio], axis=1)
            experiment_physio.drop(columns=columns_to_drop, inplace=True)

            # calculate the hrv of the baseline
            baseline_hrv = nk.hrv(baseline_physio["ECG_R_Peaks"], sampling_rate=1000)

            # Obstacle Trigger Times
            participant_takeover_times = exp2_takeover_times[
                exp2_takeover_times["subject_id"] == participant
            ].copy()
            participant_takeover_times.iloc[:, 1:] = participant_takeover_times.iloc[:, 1:].apply(
                pd.to_timedelta, args=("s",), errors="coerce"
            )

            participant_obstacle_data = pd.DataFrame()
            participant_obstacle_data_length = 0

            obstacles = ["Deer", "Cone", "Frog", "Can"]
            for obstacle in obstacles:
                # print(obstacle)
                # Participant Takeover Time for the Obstacle

                # Obstacle Trigger Time
                obstacle_trigger_time = pd.to_timedelta(
                    participant_takeover_times[f"{obstacle}TOR"].values[0], unit="s"
                )
                minute_before_obstacle = obstacle_trigger_time - pd.Timedelta(seconds=60)

                # If the obstacle trigger time is null, skip the obstacle
                if pd.isnull(obstacle_trigger_time):
                    continue
                if pd.isnull(minute_before_obstacle):
                    continue

                # Observations 1 minute before and after the obstacle
                driving_observations_before_obstacle = experiment_physio.loc[
                    minute_before_obstacle - pd.Timedelta(seconds=4) : minute_before_obstacle
                ].copy()
                driving_observations_after_obstacle = experiment_physio.loc[
                    minute_before_obstacle : minute_before_obstacle + pd.Timedelta(seconds=8)
                ].copy()

                # Observations 3 seconds before and after the obstacle
                takeover_observations_before_obstacle = experiment_physio.loc[
                    obstacle_trigger_time - pd.Timedelta(seconds=4) : obstacle_trigger_time
                ].copy()
                takeover_observations_after_obstacle = experiment_physio.loc[
                    obstacle_trigger_time : obstacle_trigger_time + pd.Timedelta(seconds=8)
                ].copy()

                # Debugging statements
                # print(f"Participant: {participant}, Obstacle: {obstacle}")
                # print(f"Driving observations before obstacle: {len(driving_observations_before_obstacle)}")
                # print(f"Driving observations after obstacle: {len(driving_observations_after_obstacle)}")
                # print(f"Takeover observations before obstacle: {len(takeover_observations_before_obstacle)}")
                # print(f"Takeover observations after obstacle: {len(takeover_observations_after_obstacle)}")

                # Check if the last observation of before obstacle is the same as the first observation of after obstacle
                if (
                    len(driving_observations_before_obstacle) > 0
                    and len(driving_observations_after_obstacle) > 0
                    and driving_observations_before_obstacle.tail(1).index
                    == driving_observations_after_obstacle.head(1).index
                ):
                    # drop the first observation of after obstacle
                    driving_observations_after_obstacle = driving_observations_after_obstacle.iloc[
                        1:
                    ]

                if (
                    len(takeover_observations_before_obstacle) > 0
                    and len(takeover_observations_after_obstacle) > 0
                    and takeover_observations_before_obstacle.tail(1).index
                    == takeover_observations_after_obstacle.head(1).index
                ):
                    # drop the first observation of after obstacle
                    takeover_observations_after_obstacle = (
                        takeover_observations_after_obstacle.iloc[1:]
                    )
                    # print("dropped")
                    # print("new length", len(takeover_observations_after_obstacle))

                # Check if the length of the observations is 3000
                if len(driving_observations_before_obstacle) > 4000:
                    # drop the first n rows
                    n = len(driving_observations_before_obstacle) - 4000
                    driving_observations_before_obstacle = (
                        driving_observations_before_obstacle.iloc[n:]
                    )
                elif len(driving_observations_before_obstacle) < 4000:
                    continue

                if len(driving_observations_after_obstacle) > 8000:
                    # drop the last n rows
                    driving_observations_after_obstacle = driving_observations_after_obstacle.iloc[
                        :8000
                    ]
                elif len(driving_observations_after_obstacle) < 8000:
                    continue

                if len(takeover_observations_before_obstacle) > 4000:
                    # drop the first n rows
                    n = len(takeover_observations_before_obstacle) - 4000
                    takeover_observations_before_obstacle = (
                        takeover_observations_before_obstacle.iloc[n:]
                    )
                elif len(takeover_observations_before_obstacle) < 4000:
                    continue

                if len(takeover_observations_after_obstacle) > 8000:
                    # drop the last n rows
                    takeover_observations_after_obstacle = (
                        takeover_observations_after_obstacle.iloc[:8000]
                    )
                elif len(takeover_observations_after_obstacle) < 8000:
                    continue

                # HRV
                driving_hrv_before_obstacle = nk.hrv_time(
                    driving_observations_before_obstacle["ECG_R_Peaks"], sampling_rate=1000
                )
                driving_hrv_after_obstacle = nk.hrv_time(
                    driving_observations_after_obstacle["ECG_R_Peaks"], sampling_rate=1000
                )

                takeover_hrv_before_obstacle = nk.hrv_time(
                    takeover_observations_before_obstacle["ECG_R_Peaks"], sampling_rate=1000
                )
                takeover_hrv_after_obstacle = nk.hrv_time(
                    takeover_observations_after_obstacle["ECG_R_Peaks"], sampling_rate=1000
                )

                if "ECG_R_Peaks" in columns_to_drop:
                    # drop the ecg_r_peaks from the observations
                    driving_observations_before_obstacle.drop(columns=["ECG_R_Peaks"], inplace=True)
                    driving_observations_after_obstacle.drop(columns=["ECG_R_Peaks"], inplace=True)

                    takeover_observations_before_obstacle.drop(columns=["ECG_R_Peaks"], inplace=True)
                    takeover_observations_after_obstacle.drop(columns=["ECG_R_Peaks"], inplace=True)

                # add the hrv features to the observations
                for column in [
                    "HRV_MeanNN",
                    "HRV_SDNN",
                    "HRV_RMSSD",
                    "HRV_CVSD",
                    "HRV_MedianNN",
                    "HRV_MadNN",
                    "HRV_MCVNN",
                    "HRV_IQRNN",
                    "HRV_SDRMSSD",
                    "HRV_Prc20NN",
                    "HRV_Prc80NN",
                    "HRV_pNN50",
                    "HRV_pNN20",
                    "HRV_MinNN",
                    "HRV_MaxNN",
                    "HRV_HTI",
                    "HRV_TINN",
                ]:
                    driving_observations_before_obstacle = (
                        driving_observations_before_obstacle.assign(
                            **{
                                column: driving_hrv_before_obstacle[column].values[0]
                                - baseline_hrv[column].values[0]
                            }
                        )
                    )
                    driving_observations_after_obstacle = (
                        driving_observations_after_obstacle.assign(
                            **{
                                column: driving_hrv_after_obstacle[column].values[0]
                                - baseline_hrv[column].values[0]
                            }
                        )
                    )
                    takeover_observations_before_obstacle = (
                        takeover_observations_before_obstacle.assign(
                            **{
                                column: takeover_hrv_before_obstacle[column].values[0]
                                - baseline_hrv[column].values[0]
                            }
                        )
                    )
                    takeover_observations_after_obstacle = (
                        takeover_observations_after_obstacle.assign(
                            **{
                                column: takeover_hrv_after_obstacle[column].values[0]
                                - baseline_hrv[column].values[0]
                            }
                        )
                    )

                # Combine the observations
                driving_observations = pd.concat(
                    [driving_observations_before_obstacle, driving_observations_after_obstacle]
                )
                takeover_observations = pd.concat(
                    [takeover_observations_before_obstacle, takeover_observations_after_obstacle]
                )

                # sliding window with step size
                resampled_data = pd.DataFrame()
                for column in driving_observations.columns:
                    if column in columns_to_normalize:
                        resampled_data[column] = (
                            driving_observations[column].rolling(window_size).mean()
                        )
                        resampled_data[column] = resampled_data[column].resample(step_size).mean()
                        resampled_data.dropna(inplace=True)
                        # resampled_data[column] = resampled_data[column].interpolate(method="ffill")
                    else:
                        resampled_data[column] = (
                            driving_observations[column].rolling(window_size).max()
                        )
                        resampled_data[column] = resampled_data[column].resample(step_size).max()
                        resampled_data.dropna(inplace=True)
                        # resampled_data[column] = resampled_data[column].interpolate(method="ffill")
                driving_observations = resampled_data

                resampled_data = pd.DataFrame()
                for column in takeover_observations.columns:
                    if column in columns_to_normalize:
                        resampled_data[column] = (
                            takeover_observations[column].rolling(window_size).mean()
                        )
                        resampled_data[column] = resampled_data[column].resample(step_size).mean()
                        resampled_data.dropna(inplace=True)
                        # resampled_data[column] = resampled_data[column].interpolate(method="ffill")
                    else:
                        resampled_data[column] = (
                            takeover_observations[column].rolling(window_size).max()
                        )
                        resampled_data[column] = resampled_data[column].resample(step_size).max()
                        resampled_data.dropna(inplace=True)
                        # resampled_data[column] = resampled_data[column].interpolate(method="ffill")
                takeover_observations = resampled_data

                # add the tot to the observations
                driving_observations = driving_observations.assign(takeover=0)
                takeover_observations = takeover_observations.assign(takeover=1)

                if participant_obstacle_data_length == 0:
                    participant_obstacle_data_length = len(driving_observations)

                if (
                    len(driving_observations)
                    != len(takeover_observations)
                    != participant_obstacle_data_length
                ):
                    print(f"Participant: {participant}, Obstacle: {obstacle}, Lengths do not match")
                    continue

                # add the observations to the participant obstacle data
                participant_obstacle_data = pd.concat(
                    [participant_obstacle_data, driving_observations, takeover_observations]
                )

            # After looping through the obstacles
            # Standardize the data
            time = participant_obstacle_data.index
            columns = participant_obstacle_data.columns
            scaler = StandardScaler()
            participant_obstacle_data[columns_to_normalize] = scaler.fit_transform(
                participant_obstacle_data[columns_to_normalize]
            )
            participant_obstacle_data = pd.DataFrame(participant_obstacle_data, columns=columns)
            participant_obstacle_data["Time"] = time
            participant_obstacle_data = participant_obstacle_data.set_index("Time")

            # seperate the data into slow and fast takeovers
            while len(participant_obstacle_data) > 0:
                # get the first observations
                observations = participant_obstacle_data.iloc[:participant_obstacle_data_length]
                participant_obstacle_data = participant_obstacle_data.iloc[
                    participant_obstacle_data_length:
                ]

                # takover from the observations
                takeover = observations["takeover"].values[0]
                observations.drop(columns=["takeover"], inplace=True)

                if takeover == 0:
                    driving_observations_data.append(observations)
                else:
                    takeover_observations_data.append(observations)

    return driving_observations_data, takeover_observations_data

In [ ]:
def accuracy( driving_observations, takeover_observations, n_components_driving, n_components_takeover, n_mix_driving, n_mix_takeover, covariance_type, ):
    iterations = 100

    accuracies = []
    true_positives_list = []
    false_positives_list = []
    true_negatives_list = []
    false_negatives_list = []

    for i in range(iterations):
        if i % 10 == 0:
            print(f"Iteration: {i}")

        # split the data
        driving_observations_train, driving_observations_test = train_test_split(
            driving_observations, test_size=0.3
        )
        takeover_observations_train, takeover_observations_test = train_test_split(
            takeover_observations, test_size=0.3
        )

        # concatenate the observations
        X_driving = None
        X_driving_lengths = []
        for data in driving_observations_train:
            if X_driving is None:
                X_driving = data.values
            else:
                X_driving = np.concatenate((X_driving, data.values))
            X_driving_lengths.append(len(data))

        X_takeover = None
        X_takeover_lengths = []
        for data in takeover_observations_train:
            if X_takeover is None:
                X_takeover = data.values
            else:
                X_takeover = np.concatenate((X_takeover, data.values))
            X_takeover_lengths.append(len(data))

        # initialize and fit the models
        driving_model = hmm.GMMHMM(
            n_components=n_components_driving, n_mix=n_mix_driving, covariance_type=covariance_type
        )
        takeover_model = hmm.GMMHMM(
            n_components=n_components_takeover,
            n_mix=n_mix_takeover,
            covariance_type=covariance_type,
        )

        # fit the models
        driving_model.fit(X_driving, X_driving_lengths)
        takeover_model.fit(X_takeover, X_takeover_lengths)

        # score the models
        accuracy = 0
        tp = 0
        fp = 0
        tn = 0
        fn = 0

        for _, observation in enumerate(driving_observations_test):
            observation = observation.values

            if driving_model.score(observation) > takeover_model.score(observation):
                accuracy += 1
                tn += 1
            else:
                fn += 1

        for _, observation in enumerate(takeover_observations_test):
            observation = observation.values

            if takeover_model.score(observation) > driving_model.score(observation):
                accuracy += 1
                tp += 1
            else:
                fp += 1

        accuracy = accuracy / (len(driving_observations_test) + len(takeover_observations_test))
        accuracies.append(accuracy)

        true_positives_list.append(tp)
        false_positives_list.append(fp)
        true_negatives_list.append(tn)
        false_negatives_list.append(fn)

        if i % 10 == 0:
            print(f"Accuracy: {accuracy}")
            print(f"True Positives: {tp}")
            print(f"False Positives: {fp}")
            print(f"True Negatives: {tn}")
            print(f"False Negatives: {fn}")

    return (
        accuracies,
        true_positives_list,
        false_positives_list,
        true_negatives_list,
        false_negatives_list,
    )

---
# Collecting Observations

Seperating observations into takeover times of greater than or less than 3 seconds, with a sliding window of 3 seconds and a step size of 3 seconds.

In [ ]:
columns_to_drop = [
    ["CH1", "CH2", "CH3", "ECG_Raw", "ECG_Clean", "ECG_Quality", 
     "ECG_R_Onsets", "ECG_R_Offsets", "ECG_P_Peaks", "ECG_P_Onsets", "ECG_P_Offsets", "ECG_Q_Peaks", 
     "ECG_S_Peaks", "ECG_T_Peaks", "ECG_T_Onsets", "ECG_T_Offsets", "ECG_Phase_Atrial", "ECG_Phase_Completion_Atrial", 
     "ECG_Phase_Ventricular", "ECG_Phase_Completion_Ventricular", "RSP_Raw", "RSP_Clean", "RSP_Peaks", "RSP_Troughs", 
     "RSP_Amplitude", "RSP_Phase", "RSP_Phase_Completion", "RSP_RVT", "EDA_Raw", "EDA_Clean", 
     "EDA_Tonic", "EDA_Phasic", "SCR_Onsets", "SCR_Height", "SCR_Recovery", "RSA_P2T", "RSA_Gates"], # "Relevant Features"
    ["CH1", "CH2", "CH3", "ECG_Raw", "ECG_Clean", "RSP_Raw", "RSP_Clean", "EDA_Raw", "EDA_Clean"] # "All Features"
]

deviation_columns = [
    ["ECG_Rate", "RSP_Rate"],  # "Relevant Features"
    ["ECG_Rate", "RSP_Rate", "EDA_Rate"],  # "All Features"
]

columns_to_normalize = [ 
    ["ECG_Rate", "RSP_Rate", "HRV_MeanNN", "HRV_SDNN", "HRV_RMSSD", "HRV_CVSD", 
     "HRV_MedianNN", "HRV_MadNN", "HRV_MCVNN", "HRV_IQRNN", "HRV_SDRMSSD", "HRV_Prc20NN", 
     "HRV_Prc80NN", "HRV_pNN50", "HRV_pNN20", "HRV_MinNN", "HRV_MaxNN", "HRV_HTI", 
     "HRV_TINN"], # "Relevant Features"
     ["ECG_Rate", "ECG_Quality", "ECG_Phase_Completion_Atrial", "ECG_Phase_Completion_Ventricular", "RSP_Amplitude", "RSP_Rate", 
      "RSP_RVT", "RSP_Phase_Completion", "RSP_Symmetry_PeakTrough", "RSP_Symmetry_RiseDecay", "EDA_Tonic", "RSA_P2T", 
      "RSA_Gates", "HRV_MeanNN", "HRV_SDNN", "HRV_RMSSD", "HRV_CVSD", "HRV_MedianNN", 
      "HRV_MadNN", "HRV_MCVNN", "HRV_IQRNN", "HRV_SDRMSSD", "HRV_Prc20NN", "HRV_Prc80NN", 
      "HRV_pNN50", "HRV_pNN20", "HRV_MinNN", "HRV_MaxNN", "HRV_HTI", "HRV_TINN"] # "All Features"
]

In [ ]:
sliding_windows = {
    "3s": ["3s", "1s"],
    "1s": ["1s", "500ms"],
    "500ms": ["500ms", "100ms"],
    "100ms": ["100ms", "50ms"],
}
# model parameters
n_components_slow = [1, 2, 3, 4]
n_mix_slow = [1, 2, 3, 4]
n_components_fast = [1, 2, 3, 4]
n_mix_fast = [1, 2, 3, 4]
covariance_types = ["full", "diag", "spherical"]

for i, columns in enumerate(columns_to_drop):
    for window_size in sliding_windows.keys():
        for step_size in sliding_windows[window_size]:

            # collect the observations for these parameters
            feature_type = "Relevant Features" if i == 0 else "All Features"
            print(f"Collecting {feature_type} with a window size of: {window_size} and a step size of: {step_size}")
            driving_observations, takeover_observations = collect_observations(exp2_folder_path, columns, deviation_columns[i], columns_to_normalize[i], window_size, step_size)
            print(f"Collected {len(driving_observations)} driving observations and {len(takeover_observations)} takeover observations.")

            # iterate over the model parameters
            for n_slow in n_components_slow:
                for m_slow in n_mix_slow:
                    for n_fast in n_components_fast:
                        for m_fast in n_mix_fast:
                            for cov in covariance_types:

                                # check if the results for these parameters already exist
                                if os.path.exists("results-to-thmm.csv"):
                                    results_df = pd.read_csv("results-to-thmm.csv")
                                    results = results_df[
                                        (results_df["Feature Type"] == feature_type)
                                        & (results_df["Window Size"] == window_size)
                                        & (results_df["Step Size"] == step_size)
                                        & (results_df["Components Slow"] == n_slow)
                                        & (results_df["Mixtures Slow"] == m_slow)
                                        & (results_df["Components Fast"] == n_fast)
                                        & (results_df["Mixtures Fast"] == m_fast)
                                        & (results_df["Covariance Type"] == cov)
                                    ]
                                    if not results.empty:
                                        print("Results already exist for these parameters.")
                                        continue

                                print("-------------------------------------------------")
                                print(f"Slow Components: {n_slow}, Slow Mixtures: {m_slow}, Fast Components: {n_fast}, Fast Mixtures: {m_fast}, Covariance Type: {cov}")
                                try:
                                    accuracies, true_positives_list, false_positives_list, true_negatives_list, false_negatives_list = accuracy(driving_observations, takeover_observations, n_slow, n_fast, m_slow, m_fast, cov)

                                    # Accuracy
                                    print(f"Average Accuracy: {np.mean(accuracies)}")
                                    print(f"Standard Deviation: {np.std(accuracies)}")
                                    print(f"Max Accuracy: {np.max(accuracies)}")
                                    print(f"Min Accuracy: {np.min(accuracies)}")

                                    # Find the index of the max accuracy
                                    max_accuracy_index = accuracies.index(np.max(accuracies))
                                    tp = true_positives_list[max_accuracy_index]
                                    print(f"True Positives: {tp}")
                                    fp = false_positives_list[max_accuracy_index]
                                    print(f"False Positives: {fp}")
                                    tn = true_negatives_list[max_accuracy_index]
                                    print(f"True Negatives: {tn}")
                                    fn = false_negatives_list[max_accuracy_index]
                                    print(f"False Negatives: {fn}")

                                    # check  if any of the values are zero
                                    if tp + fp == 0 or tp + fn == 0:
                                        precision = 0
                                        recall = 0
                                        f1_score = 0
                                    else:
                                        # Precision, Recall, and F1 Score
                                        precision = tp / (tp + fp)
                                        recall = tp / (tp + fn)
                                        f1_score = 2 * precision * recall / (precision + recall)
                                    print(f'Precision: {precision}, Recall: {recall}, F1 Score: {f1_score}')
                                    print("-------------------------------------------------")
                                    print()
                                    results = [feature_type, window_size, step_size, n_slow, m_slow, n_fast, m_fast, cov, np.mean(accuracies), np.std(accuracies), np.max(accuracies), np.min(accuracies), tp, fp, tn, fn, precision, recall, f1_score]
                                except:
                                    print(f"Model Parameters: {n_slow}, {m_slow}, {n_fast}, {m_fast}, {cov} failed.")
                                    results = [feature_type, window_size, step_size, n_slow, m_slow, n_fast, m_fast, cov, None, None, None, None, None, None, None, None, None, None, None]

                                results_columns = ["Feature Type", "Window Size", "Step Size", "Components Slow", "Mixtures Slow",  "Components Fast", "Mixtures Fast", "Covariance Type", "Mean Accuracy", "Standard Deviation", "Max Accuracy", "Min Accuracy", "True Positives", "False Positives", "True Negatives", "False Negatives", "Precision", "Recall", "F1 Score"]
                                # save the results
                                if os.path.exists("results-to-thmm.csv"):
                                    # see if the results file exists
                                    results_df = pd.read_csv("results-to-thmm.csv")
                                    # add the results to the file
                                    results_df = pd.concat([results_df, pd.DataFrame([results], columns=results_columns)])
                                    results_df.to_csv("results-to-thmm.csv", index=False)
                                else:
                                    # create the results file
                                    results_df = pd.DataFrame([results], columns=results_columns)
                                results_df.to_csv("results-to-thmm.csv", index=False)
